In [27]:
from pyspark.sql import SparkSession, column
spark = SparkSession.builder.master("local[1]")\
          .appName("SparkByExample")\
          .getOrCreate()
df = spark.read.csv("Demo.csv", header='True', inferSchema='True')
column_type = []
for col in df.dtypes:
    column_type.append(col[1])
    # print(col[0]+" , "+col[1])
new_data = [1,'abc',7]
for cl in range(len(column_type)):
    print(column_type[cl])
    print(type(new_data[cl]))
    # if(type(df[cl]) != type(new_data[cl])):
    #     raise Exception('Data type mismatch')
print(column_type)
newRow = spark.createDataFrame([new_data], column_header)
new_df = df.union(newRow)
new_df.show()

# columns = df.header
# newRow = spark.createDataFrame([(3,205,7)], columns)
# print(df)
# df.printSchema()


int
<class 'int'>
string
<class 'str'>
int
<class 'int'>
['int', 'string', 'int']


In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

expected_schema = StructType([
    StructField("Id", IntegerType(), nullable=True),
    StructField("Name", StringType(), nullable=True),
    StructField("Name_id", IntegerType(), nullable=True)
])

df = spark.read.csv("Demo.csv", header=True, inferSchema=True)

print('Data frame schema: ',df.schema)
print('Expected Schema: ',expected_schema)

if df.schema == expected_schema:
    print("Schema validation successful")
else:
    print("Schema validation failed.")


Data frame schema:  StructType([StructField('Id', StringType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])
Expected Schema:  StructType([StructField('Id', IntegerType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])
Schema validation failed.


In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

expected_schema = StructType([
    StructField("Id", IntegerType(), nullable=True),
    StructField("Name", StringType(), nullable=True),
    StructField("Name_id", IntegerType(), nullable=True)
])

df = spark.read.csv("Demo.csv", header=True, inferSchema=True)

print('Data frame schema: ', df.schema)
print('Expected Schema: ', expected_schema)

if df.schema == expected_schema:
    print("Schema validation successful")
else:
    mismatched_columns = set(df.schema.fieldNames()).symmetric_difference(set(expected_schema.fieldNames()))
    mismatched_rows = df.exceptAll(spark.createDataFrame([], schema=df.schema)).count()
    print("Schema validation failed.")
    print("Mismatched columns: ", mismatched_columns)
    print("Mismatched rows: ", mismatched_rows)


Data frame schema:  StructType([StructField('Id', StringType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])
Expected Schema:  StructType([StructField('Id', IntegerType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])
Schema validation failed.
Mismatched columns:  set()
Mismatched rows:  10


In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

expected_schema = StructType([
    StructField("Id", IntegerType(), nullable=True),
    StructField("Name", StringType(), nullable=True),
    StructField("Name_id", IntegerType(), nullable=True)
])

df = spark.read.csv("Demo.csv", header=True, inferSchema=True)

print('Data frame schema: ', df.schema)
print('Expected Schema: ', expected_schema)

if df.schema == expected_schema:
    print("Schema validation successful")
else:
    mismatched_columns = set(df.schema.fieldNames()).symmetric_difference(set(expected_schema.fieldNames()))

    df = df.withColumn("row_id", monotonically_increasing_id())
    expected_df = spark.createDataFrame([], schema=df.schema).withColumn("row_id", monotonically_increasing_id())
    mismatched_rows = df.exceptAll(expected_df).select("row_id").collect()

    print("Schema validation failed.")
    print("Mismatched columns: ", mismatched_columns)
    print("Mismatched rows:")
    for row in mismatched_rows:
        print("Row ID: ", row["row_id"])

# To get the specific row number, you can use the row_id as follows:
# mismatched_row_number = row["row_id"] + 1
# print("Row Number: ", mismatched_row_number)


Data frame schema:  StructType([StructField('Id', StringType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])
Expected Schema:  StructType([StructField('Id', IntegerType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])
Schema validation failed.
Mismatched columns:  set()
Mismatched rows:
Row ID:  7
Row ID:  4
Row ID:  2
Row ID:  9
Row ID:  6
Row ID:  5
Row ID:  8
Row ID:  0
Row ID:  3
Row ID:  1


In [66]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

expected_schema = StructType([
    StructField("Id", IntegerType(), nullable=True),
    StructField("Name", StringType(), nullable=True),
    StructField("Name_id", IntegerType(), nullable=True)
])

df = spark.read.csv("Demo.csv", header=True, inferSchema=True)

print('Data frame schema: ', df.schema)
print('Expected Schema: ', expected_schema)

if df.schema == expected_schema:
    print("Schema validation successful")
else:
    df.schema = StructType([
    StructField("Id", IntegerType(), nullable=True),
    StructField("Name", StringType(), nullable=True),
    StructField("Name_id", IntegerType(), nullable=True)
])
    expected_df = spark.createDataFrame([], schema=expected_schema)

    mismatched_rows = df.exceptAll(expected_df).select(monotonically_increasing_id().alias("row_number"))

    print("Schema validation failed.")
    print("Mismatched rows:")
    mismatched_rows.show()



Data frame schema:  StructType([StructField('Id', StringType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])
Expected Schema:  StructType([StructField('Id', IntegerType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])


AttributeError: property 'schema' of 'DataFrame' object has no setter

In [75]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

expected_schema = StructType([
    StructField("Id", IntegerType(), nullable=True),
    StructField("Name", StringType(), nullable=True),
    StructField("Name_id", IntegerType(), nullable=True)
])

df = spark.read.csv("Demo.csv", header=True, inferSchema=True)

print('Data frame schema: ', df.schema)
print('Expected Schema: ', expected_schema)

if df.schema == expected_schema:
    print("Schema validation successful")
else:
    mismatched_columns = set(df.schema.fieldNames()).symmetric_difference(set(expected_schema.fieldNames()))
    print("Schema validation failed.")
    print("Mismatched columns: ", mismatched_columns)
    print("Sample data from mismatched columns:")

    # Show sample data from mismatched columns
    for column_name in mismatched_columns:
        print(f"Column: {column_name}")
        df.select(column_name).show(10, truncate=False)


Data frame schema:  StructType([StructField('Id', StringType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True), StructField(' asd', IntegerType(), True)])
Expected Schema:  StructType([StructField('Id', IntegerType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])
Schema validation failed.
Mismatched columns:  {' asd'}
Sample data from mismatched columns:
Column:  asd
+----+
| asd|
+----+
|12  |
|23  |
|null|
|null|
|45  |
|null|
|null|
|null|
|null|
|null|
+----+



In [92]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

expected_schema = StructType([
    StructField("Id", IntegerType(), nullable=True),
    StructField("Name", StringType(), nullable=True),
    StructField("Name_id", IntegerType(), nullable=True)
])

df = spark.read.csv("Demo.csv", header=True, inferSchema=True)

print('Data frame schema: ', df.schema)
print('Expected Schema: ', expected_schema)

if df.schema == expected_schema:
    print("Schema validation successful")
else:
    mismatched_row = df.filter(~(df["Id"].cast(IntegerType()).isNotNull() &
                                 df["Name"].cast(StringType()).isNotNull() &
                                 df["Name_id"].cast(IntegerType()).isNotNull()))
    if  mismatched_row.count() > 0:
        print("Schema validation failed.")
        print("Mismatched row:")
        mismatched_row.show()
        mismatched_columns = set(df.schema.fieldNames()).symmetric_difference(set(expected_schema.fieldNames()))
    if  len(mismatched_columns)>0:
        print("Schema validation failed.")
        print("Mismatched columns: ", mismatched_columns)
        print("Sample data from mismatched columns:")
        # Show sample data from mismatched columns
        for column_name in mismatched_columns:
            print(f"Column: {column_name}")
            df.select(column_name).show(10, truncate=False)

# if all(col in df.columns for col in expected_schema.fieldNames()):
#     mismatched_row = df.filter(~(df["Id"].cast(IntegerType()).isNotNull() &
#                                  df["Name"].cast(StringType()).isNotNull() &
#                                  df["Name_id"].cast(IntegerType()).isNotNull()))

#     if mismatched_row.count() > 0:
#         print("Schema validation failed.")
#         print("Mismatched row:")
#         mismatched_row.show()
#     else:
#         print("Schema validation successful")
# else:
#     print("Schema validation failed. Mismatched columns.")


Data frame schema:  StructType([StructField('Id', StringType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True), StructField('asdas', IntegerType(), True)])
Expected Schema:  StructType([StructField('Id', IntegerType(), True), StructField('Name', StringType(), True), StructField('Name_id', IntegerType(), True)])
Schema validation failed.
Mismatched row:
+----+-----+-------+-----+
|  Id| Name|Name_id|asdas|
+----+-----+-------+-----+
|asdf|Sudip|      5| null|
+----+-----+-------+-----+

Schema validation failed.
Mismatched columns:  {'asdas'}
Sample data from mismatched columns:
Column: asdas
+-----+
|asdas|
+-----+
|13   |
|123  |
|43   |
|null |
|null |
|45   |
|67   |
|null |
|null |
|null |
+-----+

